In [1]:
import numpy as np
import json
from collections import Counter

In [2]:
target_entities = ['LOCATION1', 'LOCATION2']
aspects         = ['general','price','safety','transit-location']

In [3]:
with open('/content/drive/My Drive/absa_sentihood/data/sentihood-test.json', 'r') as fp:
    test_true = json.load(fp)
len(test_true)

1491

In [4]:
with open('/content/drive/My Drive/absa_sentihood/preds.jsonl', 'r') as fp:
    test_pred = json.load(fp)
len(test_pred)

1491

In [5]:
inexact_matches = []
exact_matches = []
for sample in test_pred:
    if sample['opinions'] != sample['model_pred']:
        inexact_matches.append(sample)
    else:
        exact_matches.append(sample)

print(len(inexact_matches), " predictions did not match all aspect-sentiments exactly.")
print(len(exact_matches), " predictions matched all aspect-sentiments exactly.")

465  predictions did not match all aspect-sentiments exactly.
1026  predictions matched all aspect-sentiments exactly.


In [6]:
ctr_correct = Counter()
ctr_all = Counter()

for sample in test_pred:
    for aspect_senti in sample['model_pred']:
        if aspect_senti['aspect'] in [a['aspect'] for a in sample['opinions']]:
            ctr_correct.update([aspect_senti['aspect']])

    for aspect_senti in sample['opinions']:
        ctr_all.update([aspect_senti['aspect']])

print("Number of true_positive detected aspects: \n")
for asp, num in ctr_correct.items():
    print(num, "out of", ctr_all[asp], asp)

Number of true_positive detected aspects: 

356 out of 586 general
101 out of 158 safety
214 out of 251 price
156 out of 221 transit-location


In [11]:
fp_true_pred_pairs = []
fn_true_pred_pairs = []

for sample in  inexact_matches:
  model_pred = sample['model_pred']
  annot_op = sample['opinions']
  # Check if extra aspect detected
  if len(model_pred) > len(annot_op):
    fp_true_pred_pairs.append((annot_op, model_pred))
  else:
    fn_true_pred_pairs.append((annot_op, model_pred))

print(len(fp_true_pred_pairs), "out of", len(inexact_matches), " incorrectly predicted samples have extra aspects detected / are false positives.")
print(len(fn_true_pred_pairs), "out of", len(inexact_matches), " incorrectly predicted samples have missed the true aspects / are false negatives.")

109 out of 465  incorrectly predicted samples have extra aspects detected / are false positives.
356 out of 465  incorrectly predicted samples have missed the true aspects / are false negatives.


In [18]:
ctr_false_pos = Counter()

for pair in fp_true_pred_pairs:

    # create set of tuples because dicts not hashable for finding difference
    annot_op = pair[0]
    annot_op = set([tuple(i.values()) for i in annot_op])
    model_pred = pair[1]
    model_pred = set([tuple(i.values()) for i in model_pred])

    diff = list(model_pred - annot_op)  # falsely detected
    for sample in diff:
        ctr_false_pos.update([sample[0]])

print("Total Number of false positive predictions of aspects: \n")  # Number of times these aspects were falsely detected
for asp, num in ctr_false_pos.items():
    print(num, asp)

Total Number of false positive predictions of aspects: 

39 general
52 transit-location
29 price
6 safety


In [20]:
ctr_false_neg = Counter()

for pair in fn_true_pred_pairs:

    # create set of tuples because dicts not hashable for finding difference
    annot_op = pair[0]
    annot_op = set([tuple(i.values()) for i in annot_op])
    model_pred = pair[1]
    model_pred = set([tuple(i.values()) for i in model_pred])

    diff = list(annot_op - model_pred)  # falsely detected
    for sample in diff:
        ctr_false_neg.update([sample[0]])

print("Total Number of false positive predictions of aspects: \n")   # Number of times these aspects were missed
for asp, num in ctr_false_neg.items():
    print(num, asp)

Total Number of false positive predictions of aspects: 

259 general
73 price
87 transit-location
65 safety
